In [2]:
from huggingface_hub import login
hf_token = " " # Huggingface token
login(token = hf_token)

In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [4]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from datasets import load_dataset, DatasetDict

# Load the dataset of full riscm
ds = load_dataset('caglarmert/full_riscm')

full = ds["train"]

# test   = indices [0, 3150)
test_ds = full.select(range(3150))

# validation = indices [3150, 6300)
val_ds = full.select(range(3150, 6300))

# train  = indices [6300, end)
train_ds = full.select(range(6300, len(full)))

# bundle into a DatasetDict
ds = DatasetDict({
    "test": test_ds,
    "val": val_ds,
    "train": train_ds,
})

# Load the model and the processor
model_id = "google/paligemma-3b-mix-224"
processor = AutoProcessor.from_pretrained(model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
from tqdm import tqdm

# Do the inference on full dataset
prompt = "caption en"

# Define a variable to store the predicitons
predictions = []
for i in tqdm(range(len(ds["test"])), desc="Predicting captions", unit="img"):
    print(f"Predicting a caption for sample {i + 1}")
    # Get the image
    image_file = ds["test"][i]["image"]
    inputs     = processor(image_file, prompt, return_tensors="pt")
    # Get the prediction
    output    = model.generate(**inputs, max_new_tokens=30)
    predicted = processor.decode(output[0], skip_special_tokens=True)[len(prompt):]
    # Store the predicted caption and print it
    predictions.append(predicted)
    print("The predicted caption:")
    print(repr(predicted))
    print()

Predicting captions:   0%|          | 0/3150 [00:00<?, ?img/s]You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


Predicting a caption for sample 1


Predicting captions:   0%|          | 1/3150 [00:08<7:13:23,  8.26s/img]You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


The predicted caption:
'\nA plane sits on the runway at an airport, its wings level with the ground. The runway is long and straight, with a grassy area on one'

Predicting a caption for sample 2


Predicting captions:   0%|          | 2/3150 [00:17<7:29:47,  8.57s/img]You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


...

The predicted caption:
'\nA satellite view of a beach.'

Predicting a caption for sample 3149


Predicting captions: 100%|█████████▉| 3149/3150 [5:36:48<00:05,  5.01s/img]You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


The predicted caption:
'\nA satellite image of a lake with many islands.'

Predicting a caption for sample 3150


Predicting captions: 100%|██████████| 3150/3150 [5:36:57<00:00,  6.42s/img]

The predicted caption:
'\nA satellite image of a rural area with a few lakes. The image shows a large area of green land with a few white and black features. There'



In [10]:
# Get the references
# Define a varible to store the reference captions
all_references = []
for i in range(len(ds["test"])):
    # Get the reference
    reference_per_sample = []
    for j in range(1,6):
        reference = ds["test"][i][f"caption_{j}"]
        reference_per_sample.append(reference)
        print(f"The reference caption_{j}:")
        print(repr(reference))

    print()
    all_references.append(reference_per_sample)

Streaming output truncated to the last 5000 lines.
'The sparse residential is on the grass next to some trees .'
The reference caption_4:
'Many trees are beside the sparse residential .'
The reference caption_5:
'A sparse residential area with some green trees and a brown building .'

The reference caption_1:
'There are blocks of residential areas between the jungles .'
The reference caption_2:
'In sparse residential areas there are several houses of different sizes and many green trees surrounded by grass .'
The reference caption_3:
'The sparse residential is on the grass next to some trees and a road .'
The reference caption_4:
'Several buildings are in sparse residential .'
The reference caption_5:
'A sparse residential area with three brown buildings and a black building .'

The reference caption_1:
'Two paths lead to a small black house .'
The reference caption_2:
'There are several houses of different sizes and several green trees in the sparse residential area, Surrounded by gra

In [11]:
# Check the format of the reference captions
print(all_references[:5])

[['A gray plane on the runway and the lawn beside .', 'A grey plane is on the runway by the lawn .', 'There is an airplane on the runway with a large lawn by the runway .', 'A plane is parked on the runway next to the grass .', 'There is a plane on the runway beside the grass .'], ['Three small planes parked in a line on the airport and a big plane behind them .', 'There are four aircraft on the open ground, The largest of which is three times as large as the smallest one .', 'There are many planes of different sizes in a clearing .', 'Four planes are parked on the runway .', 'Four planes of different sizes were on the marked ground .'], ['A plane parked in a line on the airport with some marks .', 'A white plane was parked on the instruction line .', 'An airplane parked in an open area with many containers next to it .', 'A plane is parked on the open space .', 'There is 1 plane on the ground marked .'], ['A small plane and a big plane parked next to boarding bridges .', 'A white plan

In [12]:
# Check the format of the predicted captions. Each sample starts with a new line
print(predictions[:5])

['\nA plane sits on the runway at an airport, its wings level with the ground. The runway is long and straight, with a grassy area on one', '\nA group of four airplanes are parked on a runway. The planes are white and have a wing on each side. The wing of the plane is white', "\nA large jetliner sits proudly on the tarmac, its powerful engines idling. The plane's tail and wing extend gracefully, while the white lines on", '\nAn aerial view of an airport with planes parked at gates. The tarmac is gray, and the planes are white. There are several planes parked at gates', '\nTwo airplanes are parked on the tarmac at an airport. The planes are white and have red stripes on their tails. The wing of the plane is visible']


In [13]:
# Try to clean the new lines from the predicted captions for a better evaluation at later stages
for i in range(len(predictions)):
    predictions[i] = predictions[i].lstrip("\n")

predictions[:5]

['A plane sits on the runway at an airport, its wings level with the ground. The runway is long and straight, with a grassy area on one',
 'A group of four airplanes are parked on a runway. The planes are white and have a wing on each side. The wing of the plane is white',
 "A large jetliner sits proudly on the tarmac, its powerful engines idling. The plane's tail and wing extend gracefully, while the white lines on",
 'An aerial view of an airport with planes parked at gates. The tarmac is gray, and the planes are white. There are several planes parked at gates',
 'Two airplanes are parked on the tarmac at an airport. The planes are white and have red stripes on their tails. The wing of the plane is visible']

In [14]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

nltk.download('punkt_tab')
nltk.download('punkt')

# Tokenize references and predictions:
tokenized_refs = [
    [nltk.word_tokenize(ref.lower()) for ref in refs]
    for refs in all_references
]

tokenized_hyps = [nltk.word_tokenize(pred.lower()) for pred in predictions]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
tokenized_refs[0]

[['a',
  'gray',
  'plane',
  'on',
  'the',
  'runway',
  'and',
  'the',
  'lawn',
  'beside',
  '.'],
 ['a', 'grey', 'plane', 'is', 'on', 'the', 'runway', 'by', 'the', 'lawn', '.'],
 ['there',
  'is',
  'an',
  'airplane',
  'on',
  'the',
  'runway',
  'with',
  'a',
  'large',
  'lawn',
  'by',
  'the',
  'runway',
  '.'],
 ['a',
  'plane',
  'is',
  'parked',
  'on',
  'the',
  'runway',
  'next',
  'to',
  'the',
  'grass',
  '.'],
 ['there',
  'is',
  'a',
  'plane',
  'on',
  'the',
  'runway',
  'beside',
  'the',
  'grass',
  '.']]

In [16]:
# Sentence-level BLEU-2
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/2, 1/2),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-2: {max_score*100:.2f}")

Example  1 BLEU-2: 21.44
Example  2 BLEU-2: 19.18
Example  3 BLEU-2: 8.91
Example  4 BLEU-2: 9.59
Example  5 BLEU-2: 11.74
Example  6 BLEU-2: 9.59
Example  7 BLEU-2: 15.54
Example  8 BLEU-2: 20.41
Example  9 BLEU-2: 15.54
Example 10 BLEU-2: 9.62
Example 11 BLEU-2: 13.13
Example 12 BLEU-2: 28.77
Example 13 BLEU-2: 40.45
Example 14 BLEU-2: 15.54
Example 15 BLEU-2: 50.03
Example 16 BLEU-2: 20.34
Example 17 BLEU-2: 15.90
Example 18 BLEU-2: 39.39
Example 19 BLEU-2: 17.62
Example 20 BLEU-2: 20.34
Example 21 BLEU-2: 20.34
Example 22 BLEU-2: 10.83
Example 23 BLEU-2: 8.97
Example 24 BLEU-2: 29.88
Example 25 BLEU-2: 17.62
Example 26 BLEU-2: 20.34
Example 27 BLEU-2: 16.61
Example 28 BLEU-2: 20.41
Example 29 BLEU-2: 14.55
Example 30 BLEU-2: 2.57
Example 31 BLEU-2: 22.74
Example 32 BLEU-2: 19.18
Example 33 BLEU-2: 19.03
Example 34 BLEU-2: 16.61
Example 35 BLEU-2: 16.61
Example 36 BLEU-2: 9.62
Example 37 BLEU-2: 14.38
Example 38 BLEU-2: 15.11
Example 39 BLEU-2: 23.97
Example 40 BLEU-2: 11.74
Example

In [17]:
# Corpus-level BLEU-2
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/2, 1/2),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-2: {corpus_score*100:.2f}")


Corpus BLEU-2: 19.31


In [18]:
# Sentence-level BLEU-3
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/3, 1/3, 1/3),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-3: {max_score*100:.2f}")

Example  1 BLEU-3: 11.80
Example  2 BLEU-3: 10.95
Example  3 BLEU-3: 3.13
Example  4 BLEU-3: 3.20
Example  5 BLEU-3: 3.67
Example  6 BLEU-3: 3.20
Example  7 BLEU-3: 11.99
Example  8 BLEU-3: 8.41
Example  9 BLEU-3: 9.52
Example 10 BLEU-3: 3.29
Example 11 BLEU-3: 10.72
Example 12 BLEU-3: 18.08
Example 13 BLEU-3: 33.13
Example 14 BLEU-3: 9.52
Example 15 BLEU-3: 36.99
Example 16 BLEU-3: 11.39
Example 17 BLEU-3: 4.49
Example 18 BLEU-3: 25.52
Example 19 BLEU-3: 13.04
Example 20 BLEU-3: 5.29
Example 21 BLEU-3: 16.43
Example 22 BLEU-3: 4.25
Example 23 BLEU-3: 3.06
Example 24 BLEU-3: 11.42
Example 25 BLEU-3: 10.35
Example 26 BLEU-3: 11.39
Example 27 BLEU-3: 9.95
Example 28 BLEU-3: 8.41
Example 29 BLEU-3: 4.33
Example 30 BLEU-3: 1.37
Example 31 BLEU-3: 17.70
Example 32 BLEU-3: 13.80
Example 33 BLEU-3: 9.95
Example 34 BLEU-3: 9.95
Example 35 BLEU-3: 9.95
Example 36 BLEU-3: 3.29
Example 37 BLEU-3: 9.04
Example 38 BLEU-3: 5.94
Example 39 BLEU-3: 16.01
Example 40 BLEU-3: 3.67
Example 41 BLEU-3: 19.4

In [19]:

# Corpus-level BLEU-3
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/3, 1/3, 1/3),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-3: {corpus_score*100:.2f}")


Corpus BLEU-3: 9.30


In [20]:
# Sentence-level BLEU-4
smooth = SmoothingFunction().method1
for i, (refs_per_sample, hyp_tok) in enumerate(zip(tokenized_refs, tokenized_hyps)):
    scores = []
    for refs_tok in refs_per_sample:
        score = sentence_bleu(
            [refs_tok],
            hyp_tok,
            weights=(1/4, 1/4, 1/4, 1/4),
            smoothing_function=smooth
        )
        scores.append(score)
    max_score = max(scores)
    print(f"Example {i+1:2d} BLEU-4: {max_score*100:.2f}")

Example  1 BLEU-4: 4.97
Example  2 BLEU-4: 4.70
Example  3 BLEU-4: 1.87
Example  4 BLEU-4: 1.87
Example  5 BLEU-4: 2.07
Example  6 BLEU-4: 1.87
Example  7 BLEU-4: 8.94
Example  8 BLEU-4: 5.61
Example  9 BLEU-4: 4.23
Example 10 BLEU-4: 1.94
Example 11 BLEU-4: 4.62
Example 12 BLEU-4: 6.84
Example 13 BLEU-4: 25.97
Example 14 BLEU-4: 4.23
Example 15 BLEU-4: 29.80
Example 16 BLEU-4: 4.84
Example 17 BLEU-4: 2.40
Example 18 BLEU-4: 15.75
Example 19 BLEU-4: 9.52
Example 20 BLEU-4: 2.72
Example 21 BLEU-4: 13.46
Example 22 BLEU-4: 2.85
Example 23 BLEU-4: 1.81
Example 24 BLEU-4: 7.39
Example 25 BLEU-4: 4.50
Example 26 BLEU-4: 4.84
Example 27 BLEU-4: 4.37
Example 28 BLEU-4: 5.61
Example 29 BLEU-4: 2.39
Example 30 BLEU-4: 1.00
Example 31 BLEU-4: 14.23
Example 32 BLEU-4: 9.93
Example 33 BLEU-4: 4.37
Example 34 BLEU-4: 4.37
Example 35 BLEU-4: 4.37
Example 36 BLEU-4: 1.94
Example 37 BLEU-4: 4.07
Example 38 BLEU-4: 3.87
Example 39 BLEU-4: 11.10
Example 40 BLEU-4: 2.07
Example 41 BLEU-4: 15.30
Example 4

In [21]:
# Corpus-level BLEU-4
# corpus_bleu expects list-of-list-of-tokens refs, and list-of-tokens hyps
corpus_score = corpus_bleu(
    tokenized_refs,
    tokenized_hyps,
    weights=(1/4, 1/4, 1/4, 1/4),
    smoothing_function=smooth
)
print(f"\nCorpus BLEU-4: {corpus_score*100:.2f}")


Corpus BLEU-4: 4.31


In [22]:
import nltk
from collections import Counter

# Ensure tokenizer
nltk.download('punkt', quiet=True)

def rouge_n(ref: str, hyp: str, n: int = 4):
    ref_toks = nltk.word_tokenize(ref.lower())
    hyp_toks = nltk.word_tokenize(hyp.lower())
    ref_ngrams = list(nltk.ngrams(ref_toks, n))
    hyp_ngrams = list(nltk.ngrams(hyp_toks, n))
    ref_counts = Counter(ref_ngrams)
    hyp_counts = Counter(hyp_ngrams)
    overlap = sum(min(ref_counts[ng], hyp_counts[ng]) for ng in ref_counts)
    recall = overlap / max(len(ref_ngrams), 1)
    precision = overlap / max(len(hyp_ngrams), 1)
    f1 = 2 * recall * precision / (recall + precision + 1e-8)
    return (recall, precision, f1)


# Compute ROUGE-2
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=2)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-2 Recall:    {recalls_per_sample[max_index]:.2f}")
    print(f"   ROUGE-2 Precision: {precisions_per_sample[max_index]:.2f}")
    print(f"   ROUGE-2 F1:        {f1s_per_sample[max_index]:.2f}\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-2 F1:        0.11

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'A roundabout with a bus and cars on it.'
   ROUGE-2 Recall:    0.00
   ROUGE-2 Precision: 0.00
   ROUGE-2 F1:        0.00

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'In this image we can see a map. On the map we can see buildings, trees, roads and vehicles.'
   ROUGE-2 Recall:    0.09
   ROUGE-2 Precision: 0.05
   ROUGE-2 F1:        0.06

REF:  'There are several moving vehicles on the road at the roundabout, There are several buildings and some trees and some lawns around the roundabout .'
HYP:  'In this image we can see a house, trees, vehicles on the road, grass and a fence.'
   ROUGE-2 Recall:    0.12
   ROUGE-2 Precision: 0.15
   ROUGE-2 F1:        0.13

REF:  'There are some brown buildings around the roundabout .'
HYP:  'An aerial view of a roundabout with a grassy a

In [23]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-2 METRICS ===")
print(f"Recall:    {avg_r:.2f}")
print(f"Precision: {avg_p:.2f}")
print(f"F1:        {avg_f:.2f}")

=== AVERAGE ROUGE-2 METRICS ===
Recall:    0.10
Precision: 0.07
F1:        0.07


In [24]:
# Compute ROUGE-3
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=3)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-3 Recall:    {recalls_per_sample[max_index]:.2f}")
    print(f"   ROUGE-3 Precision: {precisions_per_sample[max_index]:.2f}")
    print(f"   ROUGE-3 F1:        {f1s_per_sample[max_index]:.2f}\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-3 F1:        0.06

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'A roundabout with a bus and cars on it.'
   ROUGE-3 Recall:    0.00
   ROUGE-3 Precision: 0.00
   ROUGE-3 F1:        0.00

REF:  'The roundabout with three exits and entrances is in the residential area .'
HYP:  'In this image we can see a map. On the map we can see buildings, trees, roads and vehicles.'
   ROUGE-3 Recall:    0.00
   ROUGE-3 Precision: 0.00
   ROUGE-3 F1:        0.00

REF:  'There are several moving vehicles on the road at the roundabout, There are several buildings and some trees and some lawns around the roundabout .'
HYP:  'In this image we can see a house, trees, vehicles on the road, grass and a fence.'
   ROUGE-3 Recall:    0.08
   ROUGE-3 Precision: 0.11
   ROUGE-3 F1:        0.09

REF:  'The roundabout is on the grass next to trees and buildings .'
HYP:  'An aerial view of a rounda

In [25]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-3 METRICS ===")
print(f"Recall:    {avg_r:.2f}")
print(f"Precision: {avg_p:.2f}")
print(f"F1:        {avg_f:.2f}")

=== AVERAGE ROUGE-3 METRICS ===
Recall:    0.03
Precision: 0.02
F1:        0.02


In [26]:
# Compute ROUGE-4
all_recalls, all_precisions, all_f1s = [], [], []
for refs, pred in zip(all_references, predictions):
    recalls_per_sample, precisions_per_sample, f1s_per_sample = [], [], []
    for ref in refs:
        r, p, f = rouge_n(ref, pred, n=4)
        recalls_per_sample.append(r)
        precisions_per_sample.append(p)
        f1s_per_sample.append(f)

    max_score = max(f1s_per_sample)
    max_index = f1s_per_sample.index(max_score)
    all_recalls.append(recalls_per_sample[max_index])
    all_precisions.append(precisions_per_sample[max_index])
    all_f1s.append(f1s_per_sample[max_index])
    print(f"REF:  {refs[max_index]!r}")
    print(f"HYP:  {pred!r}")
    print(f"   ROUGE-4 Recall:    {recalls_per_sample[max_index]:.2f}")
    print(f"   ROUGE-4 Precision: {precisions_per_sample[max_index]:.2f}")
    print(f"   ROUGE-4 F1:        {f1s_per_sample[max_index]:.2f}\n")

Streaming output truncated to the last 5000 lines.
   ROUGE-4 F1:        0.00

REF:  'The roundabout connects three roads and a sculpture is in the middle of the roundabout .'
HYP:  'A roundabout with a bus and cars on it.'
   ROUGE-4 Recall:    0.00
   ROUGE-4 Precision: 0.00
   ROUGE-4 F1:        0.00

REF:  'The roundabout with three exits and entrances is in the residential area .'
HYP:  'In this image we can see a map. On the map we can see buildings, trees, roads and vehicles.'
   ROUGE-4 Recall:    0.00
   ROUGE-4 Precision: 0.00
   ROUGE-4 F1:        0.00

REF:  'There are several moving vehicles on the road at the roundabout, There are several buildings and some trees and some lawns around the roundabout .'
HYP:  'In this image we can see a house, trees, vehicles on the road, grass and a fence.'
   ROUGE-4 Recall:    0.04
   ROUGE-4 Precision: 0.06
   ROUGE-4 F1:        0.05

REF:  'The roundabout connects five roads and some houses are next to the roundabout .'
HYP:  'An aeri

In [27]:
# Report overall averages
avg_r = sum(all_recalls) / len(all_recalls)
avg_p = sum(all_precisions) / len(all_precisions)
avg_f = sum(all_f1s) / len(all_f1s)
print("=== AVERAGE ROUGE-4 METRICS ===")
print(f"Recall:    {avg_r:.2f}")
print(f"Precision: {avg_p:.2f}")
print(f"F1:        {avg_f:.2f}")

=== AVERAGE ROUGE-4 METRICS ===
Recall:    0.01
Precision: 0.00
F1:        0.00
